In [1]:
from tqdm import tqdm
import time
import papermill as pm
import pandas as pd
from datetime import datetime, timedelta
import selenium as se
from selenium import webdriver
import requests
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
from os.path import expanduser

In [2]:
def get_lines(date_list):
    gpp_lines =[]
    cash_lines = []
    options = se.webdriver.ChromeOptions()
#     options.add_argument('headless')
    driver = webdriver.Chrome(options=options, executable_path = os.path.expanduser("~") 
                              +'/anaconda3/bin/chromedriver')
    driver.implicitly_wait(30)
    for date in date_list:
        url = 'https://rotogrinders.com/resultsdb/site/draftkings/date/'+date+'/sport/nba/slate/'
        driver.get(url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        mydivs = soup.findAll("div", {"class": "ant-card-body"})
        gpp_line = (mydivs[2].text)
        if len(mydivs) > 3:
            cash_line = mydivs[3].text
        else:
            cash_line = gpp_line
        gpp_lines.append(float(gpp_line))
        cash_lines.append(float(cash_line))
    driver.quit()
    return gpp_lines, cash_lines

In [3]:
def get_cash_profit(predictions, gpp_line, cash_line):
    cash_sum = 0
    gpp_sum = []
    num_lineups = predictions.shape[0]
    for index, each_row in predictions.iterrows():
        if each_row['Points']>cash_line:
            cash_sum = cash_sum+1.8
    cash_proportion = (float(cash_sum)/num_lineups)
    return cash_proportion

In [7]:
def test_season(starting_money, weekly_bet, date_list):
    cash_sums = list()
    current_sum = starting_money
    lines_index = 0
    gpp_lines, cash_lines = get_lines(date_list)
    money_arr = []
    for date in tqdm(date_list, unit='Dates'):
        pm.execute_notebook('Modeling_SimCopy.ipynb', output_path = 'outputs/temp.ipynb', 
                            progress_bar = False, prediction_date = date, 
                            log_output = False)
        curr_pred = pd.read_csv('Predictions.csv')
        pm.execute_notebook('4_Lineup_Creation.ipynb', output_path = 'outputs/temp.ipynb', 
                            progress_bar = False, num_lineups_to_generate = 10, 
                            max_player_overlap = 4, log_output = False)
        lineups = pd.read_csv('DraftkingsLineup.csv')
        gpp_line = gpp_lines[lines_index]
        cash_line = cash_lines[lines_index]
        cash_proportion = get_cash_profit(lineups, gpp_line, cash_line)
        current_sum = current_sum - weekly_bet + (weekly_bet*cash_proportion)
        lines_index += 1
        money_arr.append(current_sum)
    return money_arr, cash_lines

In [5]:
def get_dates_list(dates):
    date_list = []
    for date in dates:
        if "/" in date:
            date_split = date.split("/")
        if "-" in date:
            date_split = date.split("-")
        year = date_split[0]
        if len(date_split[1]) > 1:
            month = date_split[1]
        else:
            month = date_split[1]
        if len(date_split[2]) > 1:
            day = date_split[2]
        else:
            day = date_split[2]
        date = year+'-'+month+'-'+day
        if date not in date_list:
            date_list.append(date)
    return date_list